Roll : 21226

Assignment 6 : Optimization

___

## Optimizing Model Parameters

Training a model is iterative and each iteration, the model guess the output and then calculates the error and using that it optimizes the parameters.

## Code

We will be using our FashionMNIST dataset to perform model training.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [2]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

These are adjustable parameters for training.

**Learning Rate** is for letting the model know how many updates it must do of the model parameter at each epoch.

**Batch Size** is to indicate the number of samples that go into the network before parameter updating.

**Epochs** is the \# of iterations over a dataset

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Loss function

Loss is the parameter that we need to optimize during Model Training.

To calculate the loss, we make a prediction using the input data and compare it against the actual lavbel.

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

It is used to adjust model parameters and reduce model error in each training iteration.

Here we use Stochastic Gradient Descent as our optimizer. But there are other popular ones such as AdamW.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Training and Testing

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Here, we train our model for 10 epochs

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311670  [   64/60000]
loss: 2.305948  [ 6464/60000]
loss: 2.283340  [12864/60000]
loss: 2.273497  [19264/60000]
loss: 2.269489  [25664/60000]
loss: 2.220786  [32064/60000]
loss: 2.232006  [38464/60000]
loss: 2.188468  [44864/60000]
loss: 2.182986  [51264/60000]
loss: 2.149406  [57664/60000]
Test Error: 
 Accuracy: 35.8%, Avg loss: 2.150219 

Epoch 2
-------------------------------
loss: 2.156248  [   64/60000]
loss: 2.150949  [ 6464/60000]
loss: 2.086017  [12864/60000]
loss: 2.107327  [19264/60000]
loss: 2.060691  [25664/60000]
loss: 1.983816  [32064/60000]
loss: 2.019568  [38464/60000]
loss: 1.924046  [44864/60000]
loss: 1.932546  [51264/60000]
loss: 1.866973  [57664/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 1.865588 

Epoch 3
-------------------------------
loss: 1.890614  [   64/60000]
loss: 1.862630  [ 6464/60000]
loss: 1.738919  [12864/60000]
loss: 1.795500  [19264/60000]
loss: 1.693294  [25664/60000]
loss: 1.636222  [32064/600

We have achieved a model accuracy of 70.6%

---